# [Hugging Faceを使って事前学習モデルを日本語の感情分析用にファインチューニングしてみた | DevelopersIO](https://dev.classmethod.jp/articles/huggingface-jp-text-classification/)

## 必要なライブラリのインストール

```
!pip install transformers
!pip install datasets
!pip install fugashi
!pip install ipadic
```

## データセット

Hugging Face のデータセット：[Hugging Face – The AI community building the future.](https://huggingface.co/datasets)

今回は以下のデータセットのうち、日本語のサブセットを使用します。

[tyqiangz/multilingual-sentiments · Datasets at Hugging Face](https://huggingface.co/datasets/tyqiangz/multilingual-sentiments)

In [1]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...  negative   
1  十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...  negative   
2  見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...  negative   
3  よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...  negative   
4  プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------


Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  味自体及び吸い心地は良いのだが、不良品が多過ぎる。私の場合５本のうち２本が蒸気も出ず、吸い込...  negative   
1             ホームボタン周りの気泡が全く抜けません。 返金をお願いしましたが、断られた。  negative   
2  新旧含めて4つのカーテンレールがあるのですが、使用出来るカーテンレールはありませんでした。 ...  negative   
3            予約注文でしたが、どこから特典であるpdfダウンロードすればよいのでしょうか…  negative   
4  前のレビューにもありましたが、片方が全く動きません。 返品しようにも、なんだかめんどくさいし...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------


Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  購入、貼付け後2週間もたたないうちに、上側から剥がれてきて画面から浮いた状態になってしまった...  negative   
1  以下の3点の理由により、期待はずれの粗悪品。 ①他の方のレビューにもある通り、天板の外観が掲...  negative   
2  この商品の内容等確認した購入したのですが、そのとおりなかなか設定ができなく、知人にもお願いし...  negative   
3  テストした結果、4000mahのスマホ一回と30％分、スマホのバッテリー残量の表示が正しけれ...  negative   
4  前回の黒いドレッサーバッグの評判がよかったため、予約して購入しました。 ガッカリです。 ヨレ...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------
Dataset multilingual-sentiments downloaded and prepared to /root/.cache/huggingface/datasets/tyqiangz___multilingual-sentiments/japanese/1.0.0/b7cdd8874d82679e59432edf79e074f595c4ad26d2e562eba4fb55f361691b07. Subsequent calls will reuse this dat

  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})

取得したデータセットは以下のようにフォーマットを設定することで、データフレームとして扱うことも可能です。

In [3]:
dataset.set_format(type="pandas")
train_df = dataset["train"][:]
train_df.head(5)

,text,source,label
0,普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...,amazon_reviews_multi,2
1,十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...,amazon_reviews_multi,2
2,見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...,amazon_reviews_multi,2
3,よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...,amazon_reviews_multi,2
4,プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...,amazon_reviews_multi,2


どうやらamazonのレビューデータが元になって、そちらに対してラベルが付与されているようです。

`source` と `label` の内訳を見てみましょう。

In [4]:
train_df.value_counts(["source", "label"])

source                label
amazon_reviews_multi  0        40000
                      1        40000
                      2        40000
dtype: int64

各ラベルの意味については、featuresを見れば分かるようになっています。

featuresは、各列の値についての詳細が記載してあります。

In [5]:
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['positive', 'neutral', 'negative'], id=None)}

このように、labelはClassLabelクラスとなっており、0,1,2がそれぞれ'positive','neutral','negative'に割り当てられていることが分かります。

ClassLabelクラスには、int2strというメソッドがあり、これでラベル名に変換することが可能です。

In [12]:
import numpy as np
from tqdm.auto import tqdm

tqdm.pandas()


def label_int2str(x):
    return dataset["train"].features["label"].int2str(x)


# train_df["label_name"] = train_df["label"].apply(label_int2str)


def _func(x, pbar) -> list[str]:
    result = label_int2str(int(x))
    pbar.update(1)
    return result


input_data = train_df["label"]

with tqdm(total=len(input_data)) as pbar:
    train_df["label_name"] = np.vectorize(_func)(input_data, pbar)

train_df.head()

  0%|          | 0/120000 [00:00<?, ?it/s]

,text,source,label,label_name
0,普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...,amazon_reviews_multi,2,negative
1,十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...,amazon_reviews_multi,2,negative
2,見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...,amazon_reviews_multi,2,negative
3,よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...,amazon_reviews_multi,2,negative
4,プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...,amazon_reviews_multi,2,negative


最後に、データフレームにしていたフォーマットを元に戻しておきます。

In [13]:
dataset.reset_format()

## モデルの検索

データをトークナイザで処理する前に、使用する事前学習モデルを決める必要があります。理由としては、通常事前学習モデルを作成した時と同じトークナイザを使用する必要があるためと考えられます。

モデルの検索もHugging Faceのページに準備されており、以下から検索が可能です。

<https://huggingface.co/models>

この中で、BERTの日本語版を探し、その中が比較的ダウンロード数の多い以下を使用することにします。

<https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking>

他にも様々な事前学習モデルがありますが、後述するトークナイザの精度などを確認し、問題が無さそうなものを選択しました。

## トークナイザの動作確認

In [14]:
from transformers import AutoTokenizer

model_ckpt = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [15]:
sample_text = "\
機械学習のコア部分のロジックを、定型的な実装部分から切り離して\
定義できるようなインターフェースに工夫されています。 \
そのためユーザーは、機械学習のコア部分のロジックの検討に\
集中することができます。\
"

In [16]:
sample_text_encoded = tokenizer(sample_text)
print(sample_text_encoded)

{'input_ids': [2, 2943, 4293, 5, 6759, 972, 5, 138, 17394, 11, 6, 23398, 81, 18, 6561, 972, 40, 24547, 16, 2279, 392, 124, 18, 23953, 7, 9909, 26, 20, 16, 21, 2610, 8, 59, 82, 4502, 9, 6, 2943, 4293, 5, 6759, 972, 5, 138, 17394, 5, 3249, 7, 4155, 34, 45, 14, 203, 2610, 8, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


結果はこのように、`input_ids` と `attention_mask` が含まれます。

input_idsは数字にエンコードされたトークンで、`attention_mask` は後段のモデルで有効なトークンかどうかを判別するためのマスクです。

無効なトークン（例えば、`[PAD]` など）に対しては、`attention_mask` を 0 として処理します。

トークナイザの結果は数字にエンコードされているため、トークン文字列を得るには、`convert_ids_to_tokens` を用います。

In [17]:
tokens = tokenizer.convert_ids_to_tokens(sample_text_encoded.input_ids)
print(tokens)

['[CLS]', '機械', '学習', 'の', 'コア', '部分', 'の', 'ロ', '##ジック', 'を', '、', '定型', '的', 'な', '実装', '部分', 'から', '切り離し', 'て', '定義', 'できる', 'よう', 'な', 'インターフェース', 'に', '工夫', 'さ', 'れ', 'て', 'い', 'ます', '。', 'その', 'ため', 'ユーザー', 'は', '、', '機械', '学習', 'の', 'コア', '部分', 'の', 'ロ', '##ジック', 'の', '検討', 'に', '集中', 'する', 'こと', 'が', 'でき', 'ます', '。', '[SEP]']


結果がこのように得られます。

先頭に##が付加されているものは、サブワード分割されているものです。

また、系列の開始が[CLS]、系列の終了(実際は複数系列の切れ目)が[SEP]という特殊なトークンとなっています。

トークナイザについては以下にも説明があります。

[cl-tohoku/bert-base-japanese-whole-word-masking · Hugging Face](https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking)

> The texts are first tokenized by MeCab morphological parser with the IPA dictionary and then split into subwords by the WordPiece algorithm. The vocabulary size is 32000.

トークン化にIPA辞書を使ったMecabが使用され、サブワード分割にはWordPieceアルゴリズムが使われているようです。

その他、文字列を再構成するには、convert_tokens_to_stringを用います。

In [18]:
decode_text = tokenizer.convert_tokens_to_string(tokens)
print(decode_text)

[CLS] 機械 学習 の コア 部分 の ロジック を 、 定型 的 な 実装 部分 から 切り離し て 定義 できる よう な インターフェース に 工夫 さ れ て い ます 。 その ため ユーザー は 、 機械 学習 の コア 部分 の ロジック の 検討 に 集中 する こと が でき ます 。 [SEP]


## データセット全体のトークン化

データセット全体に処理を適用するには、バッチ単位で処理する関数を定義し、mapを使って実施します。

- `padding=True` でバッチ内の最も長い系列長に合うようpaddingする処理を有効にします。
- `truncation=True` で、後段のモデルが対応する最大コンテキストサイズ以上を切り捨てます。

In [19]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

参考までにモデルが対応する最大コンテキストサイズは、以下で確認ができます。

In [20]:
tokenizer.model_max_length

512

これをデータセット全体に適用します。

- `batched=True` によりバッチ化され、`batch_size=None` により全体が1バッチとなります。

In [21]:
%%time
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3000
    })
})

データセット全体に適用され、カラムが追加されていることが分かります。

token_types_idは今回使用しませんが、複数の系列がある場合に使用されます。(詳細は下記を参照)

[Glossary](https://huggingface.co/docs/transformers/glossary#token-type-ids)

サンプル単位で結果を確認したい場合は、データフレームなどを使用します。

In [23]:
import pandas as pd

sample_encoded = dataset_encoded["train"][0]
pd.DataFrame(
    [
        sample_encoded["input_ids"],
        sample_encoded["attention_mask"],
        tokenizer.convert_ids_to_tokens(sample_encoded["input_ids"]),
    ],
    ["input_ids", "attention_mask", "tokens"],
).T

,input_ids,attention_mask,tokens
0,2,1,[CLS]
1,9406,1,普段
2,3276,1,使い
3,13,1,と
4,10602,1,バイク
...,...,...,...
507,0,0,[PAD]
508,0,0,[PAD]
509,0,0,[PAD]
510,0,0,[PAD]


## 分類器の実現方法

テキスト分類のためにはここから、BERTモデルの後段に分類用のヘッドを接続する必要があります。

<img src="https://raw.githubusercontent.com/cm-nakamura-shogo/devio-image/main/huggingface-jp-text-classification/huggingface-jp-text-classification-2.png" alt="drawing" width="200"/>

接続後、テキスト分類を学習する方法に大きく２種類あります。

接続した分類用ヘッドのみを学習
BERTを含むモデル全体を学習(fine-tuning)
前者は高速な学習が可能でGPUなどが利用できない場合に選択肢になり、後者の方がよりタスクに特化できるので高精度となります。

本記事では後者のfine-tuningする方法で実装していきます。

## 分類器の実装

今回のようなテキストを系列単位で分類するタスクには、既にそれ専用のクラスが準備されており、以下で構築が可能です。

In [24]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels
).to(device)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

## トレーニングの準備

学習時に性能指標を与える必要があるため、それを関数化して定義しておきます。

In [25]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

こちらは `EvalPrediction` オブジェクトをうけとる形で実装します。

`EvalPrediciton` オブジェクトは、`predictions` と `label_ids` という属性を持つ `named_tuple` です。

そして学習用のパラメータを `TrainingArguments` クラスを用いて設定します。

In [26]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = "sample-text-classification-bert"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,
    log_level="error",
)

## トレーニングの実行

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## 推論テスト

推論結果は `predict` により得ることができます。

In [ ]:
preds_output = trainer.predict(dataset_encoded["validation"])

これを混同行列で可視化してみます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

y_preds = np.argmax(preds_output.predictions, axis=1)
y_valid = np.array(dataset_encoded["validation"]["label"])
labels = dataset_encoded["train"].features["label"].names


def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()


plot_confusion_matrix(y_preds, y_valid, labels)

positive, negativeについては9割以上で正解できていますが、neutralの判別が少し難しくなっていそうです。
またpositiveをnegativeに間違えたり、negativeをpositiveに間違えたりすることは少ないようです。

## モデル保存

保存前にラベル情報を設定しておきます。

In [ ]:
id2label = {}
for i in range(dataset["train"].features["label"].num_classes):
    id2label[i] = dataset["train"].features["label"].int2str(i)

label2id = {}
for i in range(dataset["train"].features["label"].num_classes):
    label2id[dataset["train"].features["label"].int2str(i)] = i

trainer.model.config.id2label = id2label
trainer.model.config.label2id = label2id

`save_model` で保存します。

In [ ]:
model_dir = "work/nlp_tasks/text_classification/Hugging_Faceを使って事前学習モデルを日本語の感情分析用にファインチューニングしてみた/sample-text-classification-bert"
trainer.save_model(model_dir)

保存結果は以下のようなファイル構成となります。

```txt
sample-text-classification-bert
├── config.json
├── pytorch_model.bin
├── special_tokens_map.json
├── tokenizer_config.json
├── training_args.bin
└── vocab.txt
```

モデルやトークナイザの設定ファイル、そしてメインのモデルは pytorch_model.bin として保存されているようです。

## ロードして推論

In [ ]:
new_tokenizer = AutoTokenizer.from_pretrained(model_dir)

new_model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

サンプルテキストを推論します。

In [ ]:
inputs = new_tokenizer(sample_text, return_tensors="pt")

new_model.eval()

with torch.no_grad():
    outputs = new_model(
        inputs["input_ids"].to(device),
        inputs["attention_mask"].to(device),
    )
outputs.logits

logitsを推論ラベルに変換します。

In [ ]:
y_preds = np.argmax(outputs.logits.to("cpu").detach().numpy().copy(), axis=1)


def id2label(x):
    return new_model.config.id2label[x]


y_dash = [id2label(x) for x in y_preds]
y_dash